### Imports

In [1]:
%pip install -r dependencies.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)

<module 'helper' from '/Users/clinic/Documents/Clinic/code/helper.py'>

### CONSTANTS

In [3]:
ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_FILE = "training_data.csv"
TEST_FILE = "testing_data.csv"

In [4]:
REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
REGRESSION_TYPES

1: OLS
2: LASSO
3: XGB


### Read data

In [5]:
train_df = pd.read_csv(CLEANED_DATA_PATH + TRAIN_FILE)
test_df = pd.read_csv(CLEANED_DATA_PATH + TEST_FILE)

In [6]:
train_df.head()

,wt,today,tonight,tmwam,dn3sttmwmd,dn1,dn4x1,cftorrrelstd_open_0,liqlog_open_0,llirpnxm_am_1,...,tr_md_1,tr_nt_0,tr_nt_1,tr_nt_12to16,tr_nt_17to21,tr_nt_1to3,tr_nt_4to6,tr_nt_7to11,tr_toxhr_0,rrirpnxm_nt_0
0,0.000594,-0.016720,-0.015436,-0.002487,0.011511,-0.038052,0.025530,0.231635,-0.065247,0.002597,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.015402
1,0.001446,0.001475,-0.007683,-0.001346,0.027832,-0.006712,0.029269,-0.482915,1.571073,-0.000055,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.005125
2,0.000069,-0.019774,-0.013203,-0.003210,-0.001507,-0.040861,0.005203,0.165615,-2.223915,0.000659,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,-0.006332
3,0.000120,-0.000934,-0.002772,-0.000651,0.005818,-0.004393,0.006443,-1.229984,-1.670970,-0.000372,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.001978
4,0.000892,0.004249,0.002662,-0.000931,-0.034163,0.007955,-0.037853,-1.326470,0.342333,-0.000857,...,0.0,0.0,0.0,0.0,0.012116,0.0,0.0,-0.055974,0.0,0.003413


### Interaction Terms

In [15]:
col_pairs = [
    ['relvol_lsthrx15_0', 'rrirpnxm_lst15_0', 'flsdfkjslfjsldfjsldfj'],
    ["rvdelta_toxhr_0", "rrirpnxm_toxhr_0"],
    ["rvdelta_am_1", "rrirpnxm_am_1"],
    ["rvdelta_md_1", "rrirpnxm_md_1"],
    ["rvdelta_pm_1", "rrirpnxm_pm_1"],
    ["rvdelta_nt_1", "rrirpnxm_nt_1"],
    ["rvdelta_dy_1to3", "rrirpnxm_dy_1to3"],
    ["rvdelta_nt_1to3", "rrirpnxm_1to3"],
    ["rvdelta_dy_4to6", "rrirpnxm_4to6"],
    ["rvdelta_nt_4to6", "rrirpnxm_nt_4to6"],
    ["rvdelta_dy_7to11", "rrirpnxm_dy_7to11"],
    ["rvdelta_nt_7to11", "rrirpnxm_nt_7to11"],
    ["rvdelta_dy_12to16", "rrirpnxm_dy_12to16"],
    ["rvdelta_nt_12to16", "rrirpnxm_nt_12to16"],
    ["rvdelta_dy_17to21", "rrirpnxm_dy_17to21"],
    ["rvdelta_nt_17to21", "rrirpnxm_nt_17to21"]
]
interaction_terms_train_df = helper.get_df_with_interaction_terms(train_df, col_pairs)
interaction_terms_test_df = helper.get_df_with_interaction_terms(test_df, col_pairs)

[['relvol_lsthrx15_0']] missing or already been grouped!
[['rvdelta_pm_1']] missing or already been grouped!
[['rvdelta_nt_1']] missing or already been grouped!
[['rvdelta_nt_1to3']] missing or already been grouped!
[['rvdelta_dy_4to6']] missing or already been grouped!
[['rvdelta_nt_4to6']] missing or already been grouped!
[['rvdelta_nt_7to11']] missing or already been grouped!
[['rvdelta_nt_12to16']] missing or already been grouped!
[['rvdelta_nt_17to21']] missing or already been grouped!
[['relvol_lsthrx15_0']] missing or already been grouped!
[['rvdelta_pm_1']] missing or already been grouped!
[['rvdelta_nt_1']] missing or already been grouped!
[['rvdelta_nt_1to3']] missing or already been grouped!
[['rvdelta_dy_4to6']] missing or already been grouped!
[['rvdelta_nt_4to6']] missing or already been grouped!
[['rvdelta_nt_7to11']] missing or already been grouped!
[['rvdelta_nt_12to16']] missing or already been grouped!
[['rvdelta_nt_17to21']] missing or already been grouped!


### Transform the data if needed

### Run regressions

#### Ordinary Least Squares

In [8]:
ols_regression_model = helper.Regression('OLS')
model_attributes = ols_regression_model.execute(train_df, RESPONSE_NAME, test_df)
model_attributes

array([ 8.98398644e-01, -1.41553194e-02,  4.75849747e-03, -1.89818116e-02,
       -2.79251550e-02, -6.83896793e-03,  2.92836438e-02,  3.00144034e-04,
       -3.42392038e-04, -1.35224361e-01,  1.25140031e-02,  9.04946771e-03,
       -4.37903525e-02, -4.00993623e-02, -5.24000562e-04,  1.67262345e-01,
        2.64801316e-01,  2.37708240e-01,  1.39049324e+00, -1.16287146e-01,
       -7.10573558e-03,  3.20561818e-02,  8.72334414e-02,  4.95293313e-02,
       -5.67269378e-02,  1.38687981e-02,  2.15864964e-04, -1.90661792e-05,
        8.76847132e-06, -6.91134061e-05, -1.52484256e-05, -2.90190211e-05,
       -1.53536623e-04,  4.90236493e-06,  1.20645219e-04, -1.43783880e-06,
       -6.06338300e-06,  5.33123824e-05,  1.71898941e-04, -1.81140725e-05,
        4.08721746e-03, -5.69108777e-03, -4.04931808e-03, -4.65933688e-03,
       -7.29766619e-03, -3.97313407e-03, -1.49425444e-02, -2.20696623e-02,
        6.44385918e-02,  2.47113051e-01, -2.37536449e-02,  3.94236903e-03,
       -8.53573206e-04,  

In [9]:
ols_regression_model.get_metric()

1. Weighted Correlation:
[[1.         0.72854457]
 [0.72854457 1.        ]]

2. Weighted Mean Return:
0.005221154716981131

3. Weighted Scale Factor:
[0.62839211]



#### OLS w/ Interacting Terms

In [10]:
ols_interacting_model = helper.Regression('OLS')
ols_interacting_model.execute(interaction_terms_train_df, RESPONSE_NAME, interaction_terms_test_df)
ols_interacting_model.get_metric()

1. Weighted Correlation:
[[1.         0.73491725]
 [0.73491725 1.        ]]

2. Weighted Mean Return:
0.005242186620926244

3. Weighted Scale Factor:
[0.63591755]



#### LASSO

In [11]:
lasso_model = helper.Regression('LASSO')
lasso_model.execute(train_df, RESPONSE_NAME, test_df)
lasso_model.get_metric()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0401833624877621e-05, tolerance: 7.8081290168e-06
  model = cd_fast.enet_coordinate_descent_gram(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.5080694954108798e-05, tolerance: 7.8081290168e-06
  model = cd_fast.enet_coordinate_descent_gram(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4147997432035347e-05, tolerance: 7.8081290168e-06
  model = cd_fast.e

1. Weighted Correlation:
[[1.        0.7379731]
 [0.7379731 1.       ]]

2. Weighted Mean Return:
0.005155882675814751

3. Weighted Scale Factor:
[1.0640234]



### XGBoost



In [13]:
xgb_model = helper.Regression('XGB')
xgb_model.execute(train_df, RESPONSE_NAME, test_df)
xgb_model.get_metric()

1. Weighted Correlation:
[[1.         0.73367871]
 [0.73367871 1.        ]]

2. Weighted Mean Return:
0.005332684048027444

3. Weighted Scale Factor:
[0.63871306]



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:726: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)
